In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [11]:
from datetime import datetime

# import dataset for tracking

In [3]:
tracking_NO = pd.read_csv(r"C:/Users/rhoda/Desktop/Tracking_Project/10-29-2020.csv", encoding='windows-1252' )
tracking_NO

,CAP#,BOL#,Revised ETA,20F,40F,40HQ,TTL CTN,TTL Received
0,STX32-005,EGLV143999593234,02/27/2020,0,0,0,0,0
1,SIL32-002,DHL3614964533,05/13/2020,0,0,0,0,0
2,SNJ32-005,DHL3614958675,05/13/2020,0,0,0,0,0
3,SON32-009,ONEYTS0NP0864500,08/10/2020,0,0,0,0,0
4,SCA32-005,SMLMNJPK9B090600,08/11/2020,0,0,0,0,0
...,...,...,...,...,...,...,...,...
65,IL32-216,EGLV141000341839,10/30/2020,5,0,0,5,0
66,IL32-220,TOPOTTNSFOK94093/MEDUT7059099,10/30/2020,0,0,2,2,0
67,IL32-231,SMLMSHSO0B870600,10/30/2020,2,0,2,4,0
68,IL32-237,SMLMSHSO0B954200,10/30/2020,2,0,2,4,0


## cleaning and processing the dataset

### copy the dataframe to analyze and keep the original dataframe in case 

In [4]:
tracking_NO_1 = tracking_NO

### to understand how many NA ins the dataframe

In [5]:
tracking_NO_1.isna().sum() 

CAP#            0
BOL#            0
Revised ETA     0
20F             0
40F             0
40HQ            0
TTL CTN         0
TTL Received    0
dtype: int64

### show the rows that contains one or two NA in the dataframe 

In [81]:
tracking_NO_1[tracking_NO_1.isna().any(axis=1)]

,CAP#,BOL#,Revised ETA,20F,40F,40HQ,TTL CTN,TTL Received


### drop the row containing NA

In [6]:
tracking_NO_1 = tracking_NO_1.dropna()
tracking_NO_1

,CAP#,BOL#,Revised ETA,20F,40F,40HQ,TTL CTN,TTL Received
0,STX32-005,EGLV143999593234,02/27/2020,0,0,0,0,0
1,SIL32-002,DHL3614964533,05/13/2020,0,0,0,0,0
2,SNJ32-005,DHL3614958675,05/13/2020,0,0,0,0,0
3,SON32-009,ONEYTS0NP0864500,08/10/2020,0,0,0,0,0
4,SCA32-005,SMLMNJPK9B090600,08/11/2020,0,0,0,0,0
...,...,...,...,...,...,...,...,...
65,IL32-216,EGLV141000341839,10/30/2020,5,0,0,5,0
66,IL32-220,TOPOTTNSFOK94093/MEDUT7059099,10/30/2020,0,0,2,2,0
67,IL32-231,SMLMSHSO0B870600,10/30/2020,2,0,2,4,0
68,IL32-237,SMLMSHSO0B954200,10/30/2020,2,0,2,4,0


### exclude the sample item in CAP# (if there is 'S' in CAP#, then it must be the sample)

In [8]:
tracking_NO_1.drop(index=tracking_NO_1[tracking_NO_1['CAP#'].str.contains('S')].index, inplace=True)

### convert Revised ETA to datetime format 

In [33]:
# 'O' means that is string data type
tracking_NO_1['Revised ETA'].dtypes

dtype('O')

In [13]:
tracking_NO_1['Revised ETA'] =[pd.to_datetime(date) for date in tracking_NO_1['Revised ETA']] 

In [37]:
# dtype include date and time, but Today's dtype is date, then it could not be compared. Please convert datetime to date
tracking_NO_1['Revised ETA'].dtypes

dtype('<M8[ns]')

In [14]:
tracking_NO_1['Revised ETA'] = pd.to_datetime(tracking_NO_1['Revised ETA']).dt.date

In [15]:
Today = pd.Timestamp.today().date()
Today

datetime.date(2020, 10, 29)

### exclude the item whose ETA is greater than

In [16]:
# After exclude ETA, there are totally 65 items should be tracked. 
tracking_NO_1 = tracking_NO_1.drop(tracking_NO_1[tracking_NO_1["Revised ETA"]> Today].index)
tracking_NO_1

,CAP#,BOL#,Revised ETA,20F,40F,40HQ,TTL CTN,TTL Received
8,IL32-189,EGLV141000310275,2020-10-12,5,0,0,5,4
9,CA32-196,EGLV141000328557,2020-10-13,0,0,1,1,1
10,CA32-197,TOPOTTNSFOK94091/MEDUT7054777,2020-10-18,0,0,2,2,0
11,CA32-198,TOPOTTNSFOK94088/MEDUT7054751,2020-10-18,5,0,0,5,5
12,CA32-200,TOPOTTNSFOK94089/MEDUT7054769,2020-10-18,0,0,2,2,2
13,CA32-221,SMLMSHSO0B930300,2020-10-18,0,0,4,4,3
14,CA32-201,HNLTTJ20A03118/MEDUT7114779,2020-10-19,2,0,1,3,2
15,CA32-203,HNLTTJ20A03115/MEDUT7114829,2020-10-19,0,0,2,2,1
16,CA32-204,HNLTTJ20A03119/MEDUT7114787,2020-10-19,3,0,0,3,1
17,CA32-205,HNLTTJ20A03121/MEDUT7114795,2020-10-19,5,0,0,5,0


In [17]:
# Subset the data only from TOP Ocean shipping company 
tracking_TOPO = tracking_NO_1[tracking_NO_1['BOL#'].str.contains('TOPO')]
tracking_TOPO

,CAP#,BOL#,Revised ETA,20F,40F,40HQ,TTL CTN,TTL Received
10,CA32-197,TOPOTTNSFOK94091/MEDUT7054777,2020-10-18,0,0,2,2,0
11,CA32-198,TOPOTTNSFOK94088/MEDUT7054751,2020-10-18,5,0,0,5,5
12,CA32-200,TOPOTTNSFOK94089/MEDUT7054769,2020-10-18,0,0,2,2,2
29,NJ32-169,TOPOTTNSFOK94034/MEDUT2970605,2020-10-23,5,0,0,5,3
30,NJ32-170,TOPOTTNSFOK94036/MEDUT2970613,2020-10-23,5,0,0,5,3
31,NJ32-171,TOPOTTNSFOK94038/MEDUT2970621,2020-10-23,5,0,0,5,5
32,NJ32-172,TOPOTTNSFOK94065/MEDUT7001885,2020-10-23,2,0,1,3,1
34,NJ32-174,TOPOTTNSFOK94041/MEDUT2970647,2020-10-23,5,0,0,5,1
48,HCH32-302,TOPOQDOSFOK94274/MEDUQ3494745,2020-10-27,1,0,0,1,0
50,CA32-224,TOPOQDOSFOK94496/MEDUQ3637186,2020-10-28,5,0,0,5,0


In [19]:
tracking_HNLT = tracking_NO_1[tracking_NO_1['BOL#'].str.contains('HNLT')]
tracking_HNLT

,CAP#,BOL#,Revised ETA,20F,40F,40HQ,TTL CTN,TTL Received
14,CA32-201,HNLTTJ20A03118/MEDUT7114779,2020-10-19,2,0,1,3,2
15,CA32-203,HNLTTJ20A03115/MEDUT7114829,2020-10-19,0,0,2,2,1
16,CA32-204,HNLTTJ20A03119/MEDUT7114787,2020-10-19,3,0,0,3,1
17,CA32-205,HNLTTJ20A03121/MEDUT7114795,2020-10-19,5,0,0,5,0
18,CA32-206-1,HNLTTJ20A03114/MEDUT7114837,2020-10-19,0,0,2,2,1
19,CA32-206-2,HNLTTJ20A03114/MEDUT7114837,2020-10-19,0,0,0,0,0
24,IL32-204,HNLTTJ20A02663/MEDUT2913894,2020-10-22,5,0,0,5,1
28,IL32-205,HNLTTJ20A02665/MEDUT2913928,2020-10-23,3,0,0,3,2
39,IL32-187,HNLTTJ20A02837/MEDUT2965977,2020-10-25,0,0,2,2,1
42,HCH32-306,HNLTTJ20A03101/MEDUT7115800,2020-10-26,5,0,0,5,0


In [20]:
tracking_MSC = tracking_TOPO.append(tracking_HNLT)
tracking_MSC

,CAP#,BOL#,Revised ETA,20F,40F,40HQ,TTL CTN,TTL Received
10,CA32-197,TOPOTTNSFOK94091/MEDUT7054777,2020-10-18,0,0,2,2,0
11,CA32-198,TOPOTTNSFOK94088/MEDUT7054751,2020-10-18,5,0,0,5,5
12,CA32-200,TOPOTTNSFOK94089/MEDUT7054769,2020-10-18,0,0,2,2,2
29,NJ32-169,TOPOTTNSFOK94034/MEDUT2970605,2020-10-23,5,0,0,5,3
30,NJ32-170,TOPOTTNSFOK94036/MEDUT2970613,2020-10-23,5,0,0,5,3
31,NJ32-171,TOPOTTNSFOK94038/MEDUT2970621,2020-10-23,5,0,0,5,5
32,NJ32-172,TOPOTTNSFOK94065/MEDUT7001885,2020-10-23,2,0,1,3,1
34,NJ32-174,TOPOTTNSFOK94041/MEDUT2970647,2020-10-23,5,0,0,5,1
48,HCH32-302,TOPOQDOSFOK94274/MEDUQ3494745,2020-10-27,1,0,0,1,0
50,CA32-224,TOPOQDOSFOK94496/MEDUQ3637186,2020-10-28,5,0,0,5,0


In [21]:
"HNLTTJ20A03118/MEDUT7114779".find('M')

15

In [22]:
"TOPOQDOSFOK94496/MEDUQ3637186".find("M")

17

In [92]:
len(tracking_TOPO)

24

In [35]:
tracking_One_1 = tracking_TOPO[0:2]
tracking_One_1

,CAP#,BOL#,Revised ETA,20F,40F,40HQ,TTL CTN,TTL Received
10,CA32-197,TOPOTTNSFOK94091/MEDUT7054777,2020-10-18,0,0,2,2,0
11,CA32-198,TOPOTTNSFOK94088/MEDUT7054751,2020-10-18,5,0,0,5,5


In [19]:
'TOPOTTNSFOK94091/MEDUT7054777'.find('M')

17

In [38]:
df_TOPO_test = pd.DataFrame()
executable_path ='C:/Program Files (x86)/chromedriver.exe'
driver = webdriver.Chrome(executable_path)
for i in tracking_One_1['BOL#']:
    driver.get("https://www.msc.com/track-a-shipment?agencyPath=usa")
    search = driver.find_element_by_name("ctl00$ctl00$plcMain$plcMain$TrackSearch$txtBolSearch$TextField")
    search.send_keys(i[18:])
    search.send_keys(Keys.RETURN)
    time.sleep(5)
    #soup = BeautifulSoup(driver.page_source, 'html.parser')
    df_list = pd.read_html(driver.page_source)
    n = len(df_list)
    for k in range(1,len(df_list)):
        if k % 2 == 0:
            container_status = pd.DataFrame(df_list[k][0:1])
            container_status['BOL#'] =i 
            df_TOPO_test = df_TOPO_test.append(container_status)

In [39]:
df_TOPO_test

,Location,Description,Date,Vessel,Voyage,BOL#
0,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N,TOPOTTNSFOK94091/MEDUT7054777
0,"LONG BEACH, CA, US",Gate Out Full,29/10/2020,NaN,NaN,TOPOTTNSFOK94091/MEDUT7054777
0,"LONG BEACH, CA, US",Gate Out Full,26/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
0,"LONG BEACH, CA, US",Empty in Container Yard,28/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
0,"LONG BEACH, CA, US",Gate Out Full,28/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
0,"LONG BEACH, CA, US",Empty in Container Yard,27/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
0,"LONG BEACH, CA, US",Gate Out Full,28/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751


In [68]:
# container_status = pd.DataFrame(df_list[2][1:2])

,Location,Description,Date,Vessel,Voyage
1,"LONG BEACH, CA, US",Discharged,20/10/2020,MSC DANIT,039N


In [76]:
#df_TOPO

,Location,Description,Date,Vessel,Voyage,BOL#
1,"LONG BEACH, CA, US",Customs Release,15/10/2020,MSC DANIT,039N,TOPOTTNSFOK94091/MEDUT7054777
1,"LONG BEACH, CA, US",Customs Release,15/10/2020,MSC DANIT,039N,TOPOTTNSFOK94091/MEDUT7054777
1,"LONG BEACH, CA, US",Discharged,20/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Gate Out Full,26/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Gate Out Full,24/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751


In [74]:
#df_TOPO

,Location,Description,Date,Vessel,Voyage,BOL#
1,"LONG BEACH, CA, US",Customs Release,15/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Customs Release,15/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Discharged,20/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Gate Out Full,26/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Gate Out Full,24/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751


In [69]:
df_TOPO.append(container_status)

,Location,Description,Date,Vessel,Voyage
1,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N


In [63]:
#container_status = pd.DataFrame(df_list[2][1:2])
#container_status

,Location,Description,Date,Vessel,Voyage
1,"LONG BEACH, CA, US",Discharged,20/10/2020,MSC DANIT,039N


In [36]:
#n = len(df_list)

5

In [31]:
#df_list[0]

,Departure date,Vessel,Port of load
,Port of discharge,Transhipment,Price calculation date*
0,15/09/2020,MSC ALTAIR,"TIANJINXINGANG, CN"
1,"LONG BEACH, US","NINGBO, CN",07/09/2020


In [32]:
#df_list[1]

,Type,Final POD
0,40' HIGH CUBE,"LONG BEACH, US"
1,Shipped to,Price calculation date*
2,"LONG BEACH, US",07/09/2020


In [38]:
#container_1 =df_list[2]
#container_1

,Location,Description,Date,Vessel,Voyage
0,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N
1,"LONG BEACH, CA, US",Customs Release,15/10/2020,MSC DANIT,039N
2,"LONG BEACH, CA, US",Carrier Release,13/10/2020,MSC DANIT,039N
3,"NINGBO, 33, CN",Transshipment Loaded,30/09/2020,MSC DANIT,US039N
4,"NINGBO, 33, CN",Transshipment Discharged,25/09/2020,MSC ALTAIR,FY031R
5,"TIANJINXINGANG, 12, CN",Loaded,15/09/2020,MSC ALTAIR,FY037A
6,"TIANJINXINGANG, 12, CN",Gate In Full,07/09/2020,NaN,NaN
7,"TIANJINXINGANG, 12, CN",Empty to Shipper,05/09/2020,NaN,NaN


In [45]:
#container_1[1:2]

,Location,Description,Date,Vessel,Voyage
1,"LONG BEACH, CA, US",Customs Release,15/10/2020,MSC DANIT,039N


In [34]:
#df_list[3]

,Type,Final POD
0,40' HIGH CUBE,"LONG BEACH, US"
1,Shipped to,Price calculation date*
2,"LONG BEACH, US",07/09/2020


In [35]:
#df_list[4]

,Location,Description,Date,Vessel,Voyage
0,"LONG BEACH, CA, US",Discharged,25/10/2020,MSC DANIT,039N
1,"LONG BEACH, CA, US",Customs Release,15/10/2020,MSC DANIT,039N
2,"LONG BEACH, CA, US",Carrier Release,13/10/2020,MSC DANIT,039N
3,"NINGBO, 33, CN",Transshipment Loaded,30/09/2020,MSC DANIT,US039N
4,"NINGBO, 33, CN",Transshipment Discharged,25/09/2020,MSC ALTAIR,FY031R
5,"TIANJINXINGANG, 12, CN",Loaded,15/09/2020,MSC ALTAIR,FY037A
6,"TIANJINXINGANG, 12, CN",Gate In Full,07/09/2020,NaN,NaN
7,"TIANJINXINGANG, 12, CN",Empty to Shipper,05/09/2020,NaN,NaN


### set the right BOL# to corresponding shipping company(Evergreen, COSCO, MSC, ONE Tracking, SMLINE ect.)

In [40]:
df_TOPO = pd.DataFrame()
executable_path ='C:/Program Files (x86)/chromedriver.exe'
driver = webdriver.Chrome(executable_path)
for i in tracking_TOPO['BOL#']:
    driver.get("https://www.msc.com/track-a-shipment?agencyPath=usa")
    search = driver.find_element_by_name("ctl00$ctl00$plcMain$plcMain$TrackSearch$txtBolSearch$TextField")
    search.send_keys(i[18:])
    search.send_keys(Keys.RETURN)
    time.sleep(5)
    #soup = BeautifulSoup(driver.page_source, 'html.parser')
    df_list = pd.read_html(driver.page_source)
    n = len(df_list)
    for k in range(1,len(df_list)):
        if k % 2 == 0:
            container_status = pd.DataFrame(df_list[k][0:1])
            container_status['BOL#'] =i 
            df_TOPO = df_TOPO.append(container_status)

In [41]:
df_HNLT = pd.DataFrame()
executable_path ='C:/Program Files (x86)/chromedriver.exe'
driver = webdriver.Chrome(executable_path)
for i in tracking_HNLT['BOL#']:
    driver.get("https://www.msc.com/track-a-shipment?agencyPath=usa")
    search = driver.find_element_by_name("ctl00$ctl00$plcMain$plcMain$TrackSearch$txtBolSearch$TextField")
    search.send_keys(i[16:])
    search.send_keys(Keys.RETURN)
    time.sleep(6)
    #soup = BeautifulSoup(driver.page_source, 'html.parser')
    df_list = pd.read_html(driver.page_source)
    n = len(df_list)
    for k in range(1,len(df_list)):
        if k % 2 == 0:
            container_status = pd.DataFrame(df_list[k][0:1])
            container_status['BOL#'] =i 
            df_HNLT = df_HNLT.append(container_status)

In [25]:
df_TOPO 

,Location,Description,Date,Vessel,Voyage,BOL#
1,"LONG BEACH, CA, US",Customs Release,15/10/2020,MSC DANIT,039N,TOPOTTNSFOK94091/MEDUT7054777
1,"LONG BEACH, CA, US",Discharged,25/10/2020,MSC DANIT,039N,TOPOTTNSFOK94091/MEDUT7054777
1,"LONG BEACH, CA, US",Discharged,20/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Gate Out Full,26/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Gate Out Full,24/10/2020,NaN,NaN,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N,TOPOTTNSFOK94088/MEDUT7054751
1,"LONG BEACH, CA, US",Discharged,24/10/2020,MSC DANIT,039N,TOPOTTNSFOK94089/MEDUT7054769
1,"LONG BEACH, CA, US",Gate Out Full,26/10/2020,NaN,NaN,TOPOTTNSFOK94089/MEDUT7054769
1,"NEW YORK, NY, US",Carrier Release,20/10/2020,MAERSK YUKON,038E,TOPOTTNSFOK94034/MEDUT2970605


In [28]:
df_TOPO_1 = df_TOPO[['Description','Date','BOL#']]
df_TOPO_1

,Description,Date,BOL#
1,Customs Release,15/10/2020,TOPOTTNSFOK94091/MEDUT7054777
1,Discharged,25/10/2020,TOPOTTNSFOK94091/MEDUT7054777
1,Discharged,20/10/2020,TOPOTTNSFOK94088/MEDUT7054751
1,Gate Out Full,26/10/2020,TOPOTTNSFOK94088/MEDUT7054751
1,Discharged,24/10/2020,TOPOTTNSFOK94088/MEDUT7054751
1,Gate Out Full,24/10/2020,TOPOTTNSFOK94088/MEDUT7054751
1,Discharged,24/10/2020,TOPOTTNSFOK94088/MEDUT7054751
1,Discharged,24/10/2020,TOPOTTNSFOK94089/MEDUT7054769
1,Gate Out Full,26/10/2020,TOPOTTNSFOK94089/MEDUT7054769
1,Carrier Release,20/10/2020,TOPOTTNSFOK94034/MEDUT2970605


In [42]:
df_HNLT

,Location,Description,Date,Vessel,Voyage,BOL#
0,"LONG BEACH, CA, US",Empty in Container Yard,26/10/2020,NaN,NaN,HNLTTJ20A03118/MEDUT7114779
0,"LONG BEACH, CA, US",Discharged,24/10/2020,MAERSK ALTAIR,2005E,HNLTTJ20A03118/MEDUT7114779
0,"LOS ANGELES, CA, US",Empty in Container Yard,28/10/2020,NaN,NaN,HNLTTJ20A03118/MEDUT7114779
0,"LONG BEACH, CA, US",Gate Out Full,29/10/2020,NaN,NaN,HNLTTJ20A03115/MEDUT7114829
0,"LOS ANGELES, CA, US",Empty in Container Yard,28/10/2020,NaN,NaN,HNLTTJ20A03115/MEDUT7114829
0,"LOS ANGELES, CA, US",Empty in Container Yard,27/10/2020,NaN,NaN,HNLTTJ20A03119/MEDUT7114787
0,"LONG BEACH, CA, US",Discharged,24/10/2020,MAERSK ALTAIR,2005E,HNLTTJ20A03119/MEDUT7114787
0,"LONG BEACH, CA, US",Discharged,24/10/2020,MAERSK ALTAIR,2005E,HNLTTJ20A03119/MEDUT7114787
0,"LONG BEACH, CA, US",Discharged,24/10/2020,MAERSK ALTAIR,2005E,HNLTTJ20A03121/MEDUT7114795
0,"LONG BEACH, CA, US",Gate Out Full,28/10/2020,NaN,NaN,HNLTTJ20A03121/MEDUT7114795


In [46]:
df_HNLT_1 = df_HNLT[['Description','Date','BOL#']]
df_HNLT_1

,Description,Date,BOL#
0,Empty in Container Yard,26/10/2020,HNLTTJ20A03118/MEDUT7114779
0,Discharged,24/10/2020,HNLTTJ20A03118/MEDUT7114779
0,Empty in Container Yard,28/10/2020,HNLTTJ20A03118/MEDUT7114779
0,Gate Out Full,29/10/2020,HNLTTJ20A03115/MEDUT7114829
0,Empty in Container Yard,28/10/2020,HNLTTJ20A03115/MEDUT7114829
0,Empty in Container Yard,27/10/2020,HNLTTJ20A03119/MEDUT7114787
0,Discharged,24/10/2020,HNLTTJ20A03119/MEDUT7114787
0,Discharged,24/10/2020,HNLTTJ20A03119/MEDUT7114787
0,Discharged,24/10/2020,HNLTTJ20A03121/MEDUT7114795
0,Gate Out Full,28/10/2020,HNLTTJ20A03121/MEDUT7114795


In [47]:
df_MSC = df_HNLT_1.append(df_TOPO_1)
df_MSC

,Description,Date,BOL#
0,Empty in Container Yard,26/10/2020,HNLTTJ20A03118/MEDUT7114779
0,Discharged,24/10/2020,HNLTTJ20A03118/MEDUT7114779
0,Empty in Container Yard,28/10/2020,HNLTTJ20A03118/MEDUT7114779
0,Gate Out Full,29/10/2020,HNLTTJ20A03115/MEDUT7114829
0,Empty in Container Yard,28/10/2020,HNLTTJ20A03115/MEDUT7114829
...,...,...,...
1,Carrier Release,28/10/2020,TOPOQDOSFOK94496/MEDUQ3637186
1,Estimated Time of Arrival,28/10/2020,TOPOQDOSFOK94496/MEDUQ3637186
1,Carrier Release,28/10/2020,TOPOQDOSFOK94496/MEDUQ3637186
1,Estimated Time of Arrival,28/10/2020,TOPOQDOSFOK94496/MEDUQ3637186


In [48]:
Merge_MSC = pd.merge(tracking_MSC, df_MSC, on = 'BOL#', how = 'outer')
Merge_MSC


,CAP#,BOL#,Revised ETA,20F,40F,40HQ,TTL CTN,TTL Received,Description,Date
0,CA32-197,TOPOTTNSFOK94091/MEDUT7054777,2020-10-18,0,0,2,2,0,Customs Release,15/10/2020
1,CA32-197,TOPOTTNSFOK94091/MEDUT7054777,2020-10-18,0,0,2,2,0,Discharged,25/10/2020
2,CA32-198,TOPOTTNSFOK94088/MEDUT7054751,2020-10-18,5,0,0,5,5,Discharged,20/10/2020
3,CA32-198,TOPOTTNSFOK94088/MEDUT7054751,2020-10-18,5,0,0,5,5,Gate Out Full,26/10/2020
4,CA32-198,TOPOTTNSFOK94088/MEDUT7054751,2020-10-18,5,0,0,5,5,Discharged,24/10/2020
...,...,...,...,...,...,...,...,...,...,...
83,HCH32-309,HNLTTJ20A03100/MEDUT7115792,2020-10-26,5,0,0,5,0,Empty in Container Yard,28/10/2020
84,NJ32-182,HNLTTJ20A03109/MEDUT7116303,2020-10-26,0,0,2,2,0,Discharged,28/10/2020
85,NJ32-182,HNLTTJ20A03109/MEDUT7116303,2020-10-26,0,0,2,2,0,Discharged,28/10/2020
86,IL32-227,HNLTTJ20A03113/MEDUT7114761,2020-10-28,0,0,2,2,0,Estimated Delivery to the receiver,29/10/2020


In [49]:
Merge_MSC.set_index(['CAP#', 'BOL#'], inplace = True)
Merge_MSC

Revised ETA  20F  40F  40HQ  \
CAP#       BOL#                                                         
 CA32-197   TOPOTTNSFOK94091/MEDUT7054777  2020-10-18    0    0     2   
            TOPOTTNSFOK94091/MEDUT7054777  2020-10-18    0    0     2   
 CA32-198   TOPOTTNSFOK94088/MEDUT7054751  2020-10-18    5    0     0   
            TOPOTTNSFOK94088/MEDUT7054751  2020-10-18    5    0     0   
            TOPOTTNSFOK94088/MEDUT7054751  2020-10-18    5    0     0   
...                                               ...  ...  ...   ...   
 HCH32-309  HNLTTJ20A03100/MEDUT7115792    2020-10-26    5    0     0   
 NJ32-182   HNLTTJ20A03109/MEDUT7116303    2020-10-26    0    0     2   
            HNLTTJ20A03109/MEDUT7116303    2020-10-26    0    0     2   
 IL32-227   HNLTTJ20A03113/MEDUT7114761    2020-10-28    0    0     2   
            HNLTTJ20A03113/MEDUT7114761    2020-10-28    0    0     2   

                                           TTL CTN  TTL Received  \
CAP#       BOL#                                                    
 CA32-197   TOPOTTNSFOK94091/MEDUT7054777        2             0   
            TOPOTTNSFOK94091/MEDUT7054777        2             0   
 CA32-198   TOPOTTNSFOK94088/MEDUT7054751        5             5   
            TOPOTTNSFOK94088/MEDUT7054751        5             5   
            TOPOTTNSFOK94088/MEDUT7054751        5             5   
...                                            ...           ...   
 HCH32-309  HNLTTJ20A03100/MEDUT7115792          5             0   
 NJ32-182   HNLTTJ20A03109/MEDUT7116303          2             0   
            HNLTTJ20A03109/MEDUT7116303          2             0   
 IL32-227   HNLTTJ20A03113/MEDUT7114761          2             0   
            HNLTTJ20A03113/MEDUT7114761          2             0   

                                                                  Description  \
CAP#       BOL#                                                                 
 CA32-197   TOPOTTNSFOK94091/MEDUT7054777                     Customs Release   
            TOPOTTNSFOK94091/MEDUT7054777                          Discharged   
 CA32-198   TOPOTTNSFOK94088/MEDUT7054751                          Discharged   
            TOPOTTNSFOK94088/MEDUT7054751                       Gate Out Full   
            TOPOTTNSFOK94088/MEDUT7054751                          Discharged   
...                                                                       ...   
 HCH32-309  HNLTTJ20A03100/MEDUT7115792               Empty in Container Yard   
 NJ32-182   HNLTTJ20A03109/MEDUT7116303                            Discharged   
            HNLTTJ20A03109/MEDUT7116303                            Discharged   
 IL32-227   HNLTTJ20A03113/MEDUT7114761    Estimated Delivery to the receiver   
            HNLTTJ20A03113/MEDUT7114761                    Unloaded from Rail   

                                                 Date  
CAP#       BOL#                                        
 CA32-197   TOPOTTNSFOK94091/MEDUT7054777  15/10/2020  
            TOPOTTNSFOK94091/MEDUT7054777  25/10/2020  
 CA32-198   TOPOTTNSFOK94088/MEDUT7054751  20/10/2020  
            TOPOTTNSFOK94088/MEDUT7054751  26/10/2020  
            TOPOTTNSFOK94088/MEDUT7054751  24/10/2020  
...                                               ...  
 HCH32-309  HNLTTJ20A03100/MEDUT7115792    28/10/2020  
 NJ32-182   HNLTTJ20A03109/MEDUT7116303    28/10/2020  
            HNLTTJ20A03109/MEDUT7116303    28/10/2020  
 IL32-227   HNLTTJ20A03113/MEDUT7114761    29/10/2020  
            HNLTTJ20A03113/MEDUT7114761    28/10/2020  

[88 rows x 8 columns]

In [50]:
# save the dataframe into excel 
Merge_MSC.to_excel('C:/Users/rhoda/Desktop/Tracking_Project/MSC-10-29-2020.xlsx')